In [38]:
from requests import Request, Session
import json
import pandas as pd
from datetime import datetime
import time
import folium

In [43]:
#Tworzenie tabeli na dane
tabela = pd.DataFrame()

#Tworzenie listy miast, dla których będą pobierane dane
miasta = ['Poznań','Toruń','Kraków','Łódź','Wrocław','Lublin','Zielona Góra','Warszawa','Krapkowice',
                   'Białystok','Gdańsk','Katowice','Rzeszów','Kielce','Olsztyn','Szczecin']

In [44]:
#Pobieranie nazw województw
url = 'https://api.airvisual.com/v2/states?country=Poland&key=f7d3afbe-2f04-4825-840c-83ef70196930'
response = Session().get(url)

#Konwersja nazw województw do listy
dane = json.loads(response.text)
lista_wojewodztw = pd.json_normalize(dane['data'])['state'].tolist()

#Lista miast
lista_miast = ['Poznan','Torun','Krakow','Lodz','Wroclaw','Lublin','Zielona Gora','Warsaw','Krapkowice',
               'Bialystok','Gdansk','Katowice','Rzeszow','Kielce','Szczecin']

for x in range(len(lista_miast)):
    url = 'https://api.airvisual.com/v2/city?city='+lista_miast[x]+'&state='+lista_wojewodztw[x]+'&country=Poland&key=f7d3afbe-2f04-4825-840c-83ef70196930'
    response = Session().get(url)

    dane = json.loads(response.text)
    tabela = pd.concat([tabela, pd.json_normalize(dane['data'])])

    #Wstrzyamnie pętli w celu nie przekroczenia ilości zapytań do serwera 
    time.sleep(15)  

#Usuwanie zbędnych kolumn
tabela = tabela.drop(tabela.columns[[1,2,3,7,8,9,15,16]], axis=1)

#Lista nowych nazw kolumn
nowe_nazwy_kolumn = ['Miasto', 'Współrzędne geograficzne', 'Czas pomiaru AQI','AQI','Czas pomiaru danych pogodowych','Temperatura',
                     'Ciśnienie','Wilgotność','Prędkość wiatru']

#Zmiana nazw kolumn
tabela = tabela.rename(columns = dict(zip(tabela.columns[[0,1,2,3,4,5,6,7]], nowe_nazwy_kolumn)))

#Uporządkowanie indeksów tabeli
tabela.reset_index(inplace=True, drop=True)

#Konwersja daty i godziny
for x in tabela.index:
    tabela.loc[x, 'Czas pomiaru AQI'] = tabela.loc[x]['Czas pomiaru AQI'][0:19].replace('T',' ')
    tabela.loc[x, 'Czas pomiaru danych pogodowych'] = tabela.loc[x]['Czas pomiaru danych pogodowych'][0:19].replace('T',' ')

#Zmienianie nazw miast na polskie
tabela['Miasto'] = miasta
        
print("Pobrano dane")

TypeError: 'float' object is not subscriptable

In [ ]:
tabela

In [16]:
#Tworzeni list z długościami i szerokościami gograficznymi
długość_geo = []
szerokość_geo = []

for x in tabela.index:
    długość_geo.append((tabela.loc[x, 'Współrzędne geograficzne'][0]))
    szerokość_geo.append((tabela.loc[x, 'Współrzędne geograficzne'][1]))
    
#Tworzenie listy z danymi, któe można zwizualizować na mapie
warianty = ['AQI','Temperatura','Ciśnienie','Wilgotność','Prędkość wiatru','Pogoda']

#Określenie jednostek dla poszczególnych danych
jednostki = {'AQI':'', 'Temperatura': '°C', 'Ciśnienie': 'hPa', 'Wilgotność': '%', 'Prędkość wiatru': 'm/s'}

In [29]:
#Wybór wariantu przez użytkownika
wariant = input(f"Które z tych danych chcesz zobaczyć na mapie?\n{', '.join(warianty)}\n\n").lower()
if wariant not in [wybor.lower() for wybor in warianty]:
    print("Nie posiadamy tego typu danych")
else:
    if wariant == 'aqi':
        wartości = tabela[wariant.upper()].to_list()
        wariant = wariant.upper()
    else:
        wartości = tabela[wariant.capitalize()].to_list()
        wariant = wariant.capitalize()
        
    #Przypisanie jednostki
    #jednostka = jednostki[wariant]

Które z tych danych chcesz zobaczyć na mapie?
AQI, Temperatura, Ciśnienie, Wilgotność, Prędkość wiatru, Pogoda

prędkość wiatru


KeyError: 'Prędkość wiatru'

In [27]:
#Tworzenie mapy
m = folium.Map(location=[51.9194, 19.1451], zoom_start=6)
 
for name, lat, lon, value in zip(miasta, szerokość_geo, długość_geo, wartości):
     folium.Marker(
        location=[lat, lon],
        icon=folium.DivIcon(
            html=f'<div style="font-size: 12px; background-color: blue; color: white; padding: 4px; text-align: center;">{value} {jednostka}</div>',
            icon_size=(40, 30)
        )
    ).add_to(m)

#Wyświetlenie mapy
m

In [ ]:
tabela = pd.DataFrame()
#Pobieranie nazw województw
url = 'https://api.airvisual.com/v2/states?country=Poland&key=f7d3afbe-2f04-4825-840c-83ef70196930'
response = Session().get(url)

#Konwersja nazw województw do listy
dane = json.loads(response.text)
lista_wojewodztw = pd.json_normalize(dane['data'])['state'].tolist()

#Lista miast
lista_miast = ['Poznan','Torun','Krakow','Lodz','Wroclaw','Lublin','Zielona Gora','Warsaw','Krapkowice',
               'Bialystok','Gdansk','Katowice','Rzeszow','Kielce','Szczecin']

for x in range(len(lista_miast)):
    url = 'https://api.airvisual.com/v2/city?city='+lista_miast[x]+'&state='+lista_wojewodztw[x]+'&country=Poland&key=f7d3afbe-2f04-4825-840c-83ef70196930'
    response = Session().get(url)

    dane = json.loads(response.text)
    tabela = pd.concat([tabela, pd.json_normalize(dane['data'])])

    #Wstrzyamnie pętli w celu nie przekroczenia ilości zapytań do serwera 
    time.sleep(15)  

#Usuwanie zbędnych kolumn
tabela = tabela.drop(tabela.columns[[1,2,3,7,8,9,15,16]], axis=1)

#Lista nowych nazw kolumn
nowe_nazwy_kolumn = ['Miasto', 'Współrzędne geograficzne', 'Czas pomiaru AQI','AQI','Czas pomiaru danych pogodowych','Temperatura',
                     'Ciśnienie','Wilgotność','Prędkość wiatru']

#Zmiana nazw kolumn
tabela = tabela.rename(columns = dict(zip(tabela.columns[[0,1,2,3,4,5,6,7]], nowe_nazwy_kolumn)))

#Uporządkowanie indeksów tabeli
tabela.reset_index(inplace=True, drop=True)

In [ ]:
tabela